# 1. Retriever - embeddings
* The retriever acts like an internal search engine: given the user query, it returns a few relevant snippets from your knowledge base.

These snippets will then be fed to the Reader Model to help it generate its answer.

So our objective here is, given a user question, to find the most snippets from our knowledge base to answer that question.

This is a wide objective, it leaves open some questions. How many snippets should we retrieve? This parameter will be named top_k.

How long should these snippets be? This is called the chunk size. There's no one-size-fits-all answers, but here are a few elements:

🔀 Your chunk size is allowed to vary from one snippet to the other.
Since there will always be some noise in your retrieval, increasing the top_k increases the chance to get relevant elements in your retrieved snippets. 🎯 Shooting more arrows increases your probability to hit your target.
Meanwhile, the summed length of your retrieved documents should not be too high: for instance, for most current models 16k tokens will probably drown your Reader model in information due to Lost-in-the-middle phenomenon. 🎯 Give your reader model only the most relevant insights, not a huge pile of books!
In this notebook, we use Langchain library since it offers a huge variety of options for vector databases and allows us to keep document metadata throughout the processing.

1.1 Split the documents into chunks
In this part, we split the documents from our knowledge base into smaller chunks which will be the snippets on which the reader LLM will base its answer.
The goal is to prepare a collection of semantically relevant snippets. So their size should be adapted to precise ideas: too small will truncate ideas, too large will dilute them.
💡 Many options exist for text splitting: splitting on words, on sentence boundaries, recursive chunking that processes documents in a tree-like way to preserve structure information... To learn more about chunking, I recommend you read this great notebook by Greg Kamradt.

Recursive chunking breaks down the text into smaller parts step by step using a given list of separators sorted from the most important to the least important separator. If the first split doesn't give the right size or shape chunks, the method repeats itself on the new chunks using a different separator. 

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import FAISS
import os 
import PyPDF2
import torch

# 2 Building the vector database
We want to compute the embeddings for all the chunks of our knowledge base: to learn more on sentence embeddings, we recommend reading this guide.

How does retrieval work ?
Once the chunks are all embedded, we store them into a vector database. When the user types in a query, it gets embedded by the same model previously used, and a similarity search returns the closest documents from the vector database.

The technical challenge is thus, given a query vector, to quickly find the nearest neighbours of this vector in the vector database. To do this, we need to choose two things: a distance, and a search algorithm to find the nearest neighbors quickly within a database of thousands of records.

Nearest Neighbor search algorithm
There are plentiful choices for the nearest neighbor search algorithm: we go with Facebook's FAISS, since FAISS is performant enough for most use cases, and it is well known thus widely implemented.

Distances
Regarding distances, you can find a good guide here. In short:

Cosine similarity computes similarity between two vectors as the cosinus of their relative angle: it allows us to compare vector directions are regardless of their magnitude. Using it requires to normalize all vectors, to rescale them into unit norm.
Dot product takes into account magnitude, with the sometimes undesirable effect that increasing a vector's length will make it more similar to all others.
Euclidean distance is the distance between the ends of vectors.
You can try this small exercise to check your understanding of these concepts. But once vectors are normalized, the choice of a specific distance does not matter much.

Our particular model works well with cosine similarity, so choose this distance, and we set it up both in the Embedding model, and in the distance_strategy argument of our FAISS index. With cosine similarity, we have to normalize our embeddings.

The user query's embedding is also shown : we want to find the k document that have the closest meaning, thus we pick the k closest vectors.

In [ ]:
def load_chunk_persist_pdf():
    pdf_folder_path = "/home/baskar/CALIX_LLM/RAG/PHYSICS_DATSET"
    documents = []
    for file in os.listdir(pdf_folder_path):
        if file.endswith('.pdf'):
            pdf_path = os.path.join(pdf_folder_path, file)
            loader = PyPDFLoader(pdf_path)
            documents.extend(loader.load())
    text_splitter = RecursiveCharacterTextSplitter(
                                                chunk_size=1000,
                                                chunk_overlap=40,
                                                length_function=len,
                                                is_separator_regex=False,)
    chunked_documents = text_splitter.split_documents(documents)
    model_name = "intfloat/e5-base-v2"
    encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
    embeddings = HuggingFaceBgeEmbeddings(
                                            model_name=model_name,
                                            model_kwargs={'device': 'cuda'},  #getting RuntimeError if used 'device':'cuda' due to compatibility issues
                                            encode_kwargs=encode_kwargs
                                            )
    vector_db = FAISS.from_documents(chunked_documents,embedding=embeddings)
    vector_db.save_local("home/baskar/CALIX_LLM/RAG/VECTOR_DB")
    new_db = FAISS.load_local("home/baskar/CALIX_LLM/RAG/VECTOR_DB", embeddings=embeddings, allow_dangerous_deserialization=True)
    retriever = new_db.as_retriever(search_kwargs={"k": 4})   #retrieving top 4 similar documents  
    return retriever
main_retriever = load_chunk_persist_pdf()